## Machine Predictive Maintenance
## Capstone Two - Step One: DataWrangling
This project is to analyze the correlation between machine failure modes and multiple variations. The machine failure consists of five independent failure modes. 14 features will be analyzed as variations, including temperatures, rotational speed, etc. 

The raw dataset can be obtained through the following link:

https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import os

C:\ProgramData\anaconda3\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\Dallen\AppData\Roaming\Python\Python311\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


In [2]:
# The raw data is downloaded from the website 
# https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

machine_data = pd.read_csv('../MachinePredictiveMaintenance/RawData/ai4i2020.csv')

In [3]:
machine_data.info

<bound method DataFrame.info of         UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0         1     M14860    M                298.1                    308.6   
1         2     L47181    L                298.2                    308.7   
2         3     L47182    L                298.1                    308.5   
3         4     L47183    L                298.2                    308.6   
4         5     L47184    L                298.2                    308.7   
...     ...        ...  ...                  ...                      ...   
9995   9996     M24855    M                298.8                    308.4   
9996   9997     H39410    H                298.9                    308.4   
9997   9998     M24857    M                299.0                    308.6   
9998   9999     H39412    H                299.0                    308.7   
9999  10000     M24859    M                299.0                    308.7   

      Rotational speed [rpm]  Torque [Nm]  

The dataset consists of 10000 data points stored as rows with 14 features in columns.

In [4]:
machine_data.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


Data description:  
  
1. UID: unique identifier ranging from 1 to 10000.  
2. Product ID: consisting of a letter L, M, or H as product quality variants and a variant-specific serial number.  
          L: low (50% of all products)  
          M: medium (30%)  
          H: high (20%)  
3. Air temperature, Process temperature, Rotational speed, Torque, Tool wear are the working status recorded in the machine.  
4. Machine failure: Whether the machine has failed in this particular datapoint for any of the failure modes are true.  
5. The machine failure consists of five independent failure modes. If at least one of the above failure modes is true, the process fails and the 'machine failure' label is set to 1.  
          TWF: tool wear failure  
          HDF: heat dissipation failure  
          PWF: power failure  
          OSF: overstrain failure  
          RNF: random failures

In [5]:
# Checking missing values
missing = pd.concat([machine_data.isnull().sum(), 100 * machine_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
UDI,0,0.0
Product ID,0,0.0
Type,0,0.0
Air temperature [K],0,0.0
Process temperature [K],0,0.0
Rotational speed [rpm],0,0.0
Torque [Nm],0,0.0
Tool wear [min],0,0.0
Machine failure,0,0.0
TWF,0,0.0


The dataset doesn't have any missing values.

In [6]:
# Checking data type of each column
data_types = machine_data.dtypes
print(data_types)

UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object


The data types for each column match with their physical attributes. There is no need to adjust the data type so far.

In [7]:
# Rename 'UDI' column to 'UID' and use the first column as the new index
machine_data.rename(columns={'UDI': 'UID'}, inplace=True)
machine_data = machine_data.set_index(machine_data.columns[0])

In [8]:
machine_data.head()

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
UID,,,,,,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [9]:
# Check the value counts of column 'Type'
machine_data['Type'].value_counts()

Type
L    6000
M    2997
H    1003
Name: count, dtype: int64

The percentage of L, M and H are very close the description for "Type", which is 60%, 30% and 10%.

In [10]:
# Create a dateframe named 'failure_record' 
# and check the relationships between "Machine Failure" and five single failure modes.
failure_record = machine_data[["Machine failure", "TWF", "HDF", "PWF", "OSF", "RNF"]].copy()

# Add 'total' column to record the total number of single failures per 'Machine Failure'.
failure_record['total'] = failure_record[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1)

#Check the relationship between 'Machine failure' and 'total'.
failure_record[['Machine failure', 'total']].value_counts()

Machine failure  total
0                0        9643
1                1         306
                 2          23
0                1          18
1                0           9
                 3           1
Name: count, dtype: int64

In [11]:
# Check the machine failure cases and make sure they all have at least one single failure mode.
failure_record[(failure_record != 0).any(axis=1)].sort_values(by = 'total', ascending=False)

,Machine failure,TWF,HDF,PWF,OSF,RNF,total
UID,,,,,,,
5910,1,1,0,1,1,0,3
8927,1,0,0,1,1,0,2
1325,1,0,0,1,1,0,2
1497,1,0,0,1,1,0,2
7084,1,0,0,1,1,0,2
...,...,...,...,...,...,...,...
4685,1,0,0,0,0,0,0
8507,1,0,0,0,0,0,0
9016,1,0,0,0,0,0,0


The last five observations have been recorded as machine failures, but none of the five individual failures are true, which conflicts with the data description.

In [12]:
# Update the RNF (Random Failure) from 0 to 1 for those rows 
# that have 1 in 'Machine failure' but '0' in all five single failure mode.

failure_record.loc[(failure_record['Machine failure'] == 1) & (failure_record['total'] == 0), 'RNF'] = 1

# Recaculate the total failures
failure_record['total'] = failure_record[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1)

In [13]:
# Check the machine failure cases and make sure all single failures contribute to the machine failure.
failure_record[failure_record['Machine failure'] == 0].sort_values(by = 'total', ascending=False)

,Machine failure,TWF,HDF,PWF,OSF,RNF,total
UID,,,,,,,
3066,0,0,0,0,0,1,1
2560,0,0,0,0,0,1,1
1222,0,0,0,0,0,1,1
3453,0,0,0,0,0,1,1
6092,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...
3304,0,0,0,0,0,0,0
3305,0,0,0,0,0,0,0
3306,0,0,0,0,0,0,0


In [14]:
# Update the Machine failure from 0 to 1 for those rows 
# that have at least one in single mode failure' but '0' in Machine failure.

failure_record.loc[failure_record['total'] > 0, 'Machine failure'] = 1

In [15]:
#Check the relationship between 'Machine failure' and 'total'.
failure_record[['Machine failure', 'total']].value_counts()

Machine failure  total
0                0        9643
1                1         333
                 2          23
                 3           1
Name: count, dtype: int64

The outcome is now the same as how they describe the failure columns.
"If at least one of the failure modes is true, the process fails and the 'machine failure' label is set to 1." --- Data Description.  

In [16]:
failure_record['Machine failure'].value_counts()

Machine failure
0    9643
1     357
Name: count, dtype: int64

There are 357 failures among 10000 events.

In [17]:
# update machine_data with adjusted failure_record
machine_data.update(failure_record)
machine_data.head()

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
UID,,,,,,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [18]:
tool_wear = machine_data['Tool wear [min]'].value_counts()
tool_wear

Tool wear [min]
0      120
2       69
5       63
7       58
59      58
      ... 
237      1
239      1
241      1
251      1
253      1
Name: count, Length: 246, dtype: int64

Tool wears are reset to 0 when maintenance events are finished.
120 tools were used, which means 119 maintenance events were performed.

In [19]:
profile = ProfileReport(machine_data, title="Profiling Report")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Other Variables have been summarized ProfileReport and will be analyzed for future.

Air temperature [K] : ranged from 295.3 to 304.5.
Process temperature [K] : ranged from 305.7 to 313.8
Rotational speed [rpm] : ranged from 1168 to 2886
Torque [Nm] : ranged from 3.8 to 76.6

In [20]:
machine_data.to_csv('../MachinePredictiveMaintenance/SavedData/data01.csv', index=True)